In [ ]:
import pandas as pd
project = pd.read_excel(r'C:\Users\User\Desktop\MetroHub DA\Python\Project Data.xlsx')
project_update = project[~(project['Order Date']=='Order Date')]  # clean the data by dropping the rows with the above issue
project_update['Order Date'] = pd.to_datetime(project_update['Order Date']) # converting the [Order Date] to datetime format finally
import warnings
warnings.filterwarnings('ignore')
project_clean = project_update[~(project_update.isna().any(axis=1))] # removing blank [isna] rows
project_clean['Order Month'] = project_clean['Order Date'].dt.month # Extracting month from date
# Turnong the month number to month name using a LOOP command and an IF statement 
num = (project_clean['Order Month'])
obj = [{'mnumb': 1, 'mname':'January'}, {'mnumb': 2, 'mname':'February'}, {'mnumb': 3, 'mname':'March'},
       {'mnumb': 4, 'mname':'April'}, {'mnumb': 5, 'mname':'May'}, {'mnumb': 6, 'mname':'June'},
       {'mnumb': 7, 'mname':'July'}, {'mnumb': 8, 'mname':'August'}, {'mnumb': 9, 'mname':'September'},
       {'mnumb': 10, 'mname':'October'}, {'mnumb': 11, 'mname':'November'}, {'mnumb': 12, 'mname':'December'}
]
def month_conv(num):
    for ob in obj:
        if(num == ob['mnumb']):
            return(ob['mname'])

project_clean['Order Month'] = project_clean['Order Month'].apply(month_conv)
# Creating a sales total column
project_clean['Sales'] = project_clean['Quantity Ordered'] * project_clean['Price Each']

# Calculating the total sales per month and using a bar chart for visuals
project_clean.groupby('Order Month')['Sales'].sum().sort_values(ascending=False)

import matplotlib.pyplot as plt
sales = project_clean.groupby('Order Month')['Sales'].sum().sort_values(ascending=False)
x = sales.index
y = sales.values 
plt.figure(figsize=(12,5))
plt.bar(x,y,color='b')
plt.title('Monthly Sales', color='b', fontdict={'fontsize':25, 'fontweight':'bold'})
plt.xticks(rotation=45)
plt.xlabel("Months", color='b', fontdict={'fontweight':'bold'})
plt.ylabel("Sales in Millions", color='b', fontdict={'fontweight':'bold'})
plt.show()

# calculating sales per city and adding a visual for clarity
cities = project_clean['Purchase Address'].str.split(',',expand=True)[1]
states = project_clean['Purchase Address'].str.split(',',expand=True)[2].str.split(expand=True)[0]
project_clean['Purchase City'] = cities + "(" +states + ")"
project_clean.groupby('Purchase City')['Sales'].sum().sort_values(ascending=False)

pur_city = project_clean.groupby('Purchase City')['Sales'].sum().sort_values(ascending=False)
x_city = pur_city.index
y_city = pur_city.values
plt.figure(figsize=(11,5))
plt.bar(x_city, y_city, color='purple')
plt.title("Total Sales by City", color='purple', fontdict={'fontsize':20, 'fontweight':'bold'})
plt.xlabel("City(State)", color='purple', fontdict={'fontsize':13, 'fontweight':'bold'})
plt.ylabel("Total Sales in Millions", color='purple', fontdict={'fontsize':13, 'fontweight':'bold'})
plt.xticks(rotation=45)
plt.show()

# Finding the best hour to run adverts using the data provided
hour = project_clean['Order Date'].dt.hour
project_clean["Hour"] = hour

# Method 1
def pur_hour(hour):
    if hour < 12:
        return('Morning')
    elif hour <= 15:
        return('Afternoon')
    elif hour < 20:
        return('Evening')
    else:
        return('Night')    # 0-11: Morning, 12-15: Afternoon, 16-19: Evening, and 20-23: Night
    
project_clean['Purchase Time'] = project_clean['Hour'].apply(pur_hour)
project_clean.groupby('Purchase Time')['Order ID'].count().sort_values(ascending=False)

pur_time = project_clean.groupby('Purchase Time')['Order ID'].count().sort_values(ascending=False)
Ptime = pur_time.index
tot_orders = pur_time.values

plt.figure(figsize=(13,4))
plt.pie(tot_orders, labels=Ptime, autopct='%1.0f%%', pctdistance=0.6)
plt.title("Total Orders per Time", color='red', fontdict={'fontsize':20, 'fontweight':'bold'})
plt.show()

# Method 2
hr_count = project_clean.groupby("Hour")["Order ID"].count().sort_values(ascending=True)

plt.figure(figsize=(14,5))
plt.bar(hr_count.index, hr_count.values, color='brown')
plt.title("Total Order per Hour", color='brown', fontdict={'fontsize':20, 'fontweight':'bold'})
plt.xlabel('24-hour Time', color='brown', fontdict={'fontsize':12, 'fontweight':'bold'})
plt.ylabel('Total Orders', color='brown', fontdict={'fontsize':12, 'fontweight':'bold'})
plt.show()